In [ ]:
!pip install -U transformers datasets accelerate huggingface_hub evaluate optimum[openvino,nncf] --quiet

In [ ]:
from datasets import load_dataset, concatenate_datasets, Dataset, ClassLabel, load_from_disk, load_metric
from transformers import AutoModelForSequenceClassification, DataCollatorWithPadding, TrainingArguments, Trainer, AutoTokenizer
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import transformers
import pandas as pd

In [ ]:
import sqlite3

cnx = sqlite3.connect('/kaggle/input/feedback-2252024/database.sqlite')

df = pd.read_sql_query("SELECT text_, label_ FROM feedback", cnx).rename(columns={'text_':'text', 'label_': 'label'})

In [ ]:
df

In [ ]:
df.groupby('label').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM, pipeline

pipe = pipeline("fill-mask", model="FacebookAI/xlm-roberta-large", device="cuda")

In [ ]:
pipe("زبان فارسی، از زبان های <mask> دنیا است.", top_k=2)

In [ ]:
import string

In [ ]:
stopwords_en = ['call', 'upon', 'still', 'nevertheless', 'down', 'every', 'forty', '‘re', 'always', 'whole', 'side', "n't", 'now', 'however', 'an', 'show', 'least', 'give', 'below', 'did', 'sometimes', 'which', "'s", 'nowhere', 'per', 'hereupon', 'yours', 'she', 'moreover', 'eight', 'somewhere', 'within', 'whereby', 'few', 'has', 'so', 'have', 'for', 'noone', 'top', 'were', 'those', 'thence', 'eleven', 'after', 'no', '’ll', 'others', 'ourselves', 'themselves', 'though', 'that', 'nor', 'just', '’s', 'before', 'had', 'toward', 'another', 'should', 'herself', 'and', 'these', 'such', 'elsewhere', 'further', 'next', 'indeed', 'bottom', 'anyone', 'his', 'each', 'then', 'both', 'became', 'third', 'whom', '‘ve', 'mine', 'take', 'many', 'anywhere', 'to', 'well', 'thereafter', 'besides', 'almost', 'front', 'fifteen', 'towards', 'none', 'be', 'herein', 'two', 'using', 'whatever', 'please', 'perhaps', 'full', 'ca', 'we', 'latterly', 'here', 'therefore', 'us', 'how', 'was', 'made', 'the', 'or', 'may', '’re', 'namely', "'ve", 'anyway', 'amongst', 'used', 'ever', 'of', 'there', 'than', 'why', 'really', 'whither', 'in', 'only', 'wherein', 'last', 'under', 'own', 'therein', 'go', 'seems', '‘m', 'wherever', 'either', 'someone', 'up', 'doing', 'on', 'rather', 'ours', 'again', 'same', 'over', '‘s', 'latter', 'during', 'done', "'re", 'put', "'m", 'much', 'neither', 'among', 'seemed', 'into', 'once', 'my', 'otherwise', 'part', 'everywhere', 'never', 'myself', 'must', 'will', 'am', 'can', 'else', 'although', 'as', 'beyond', 'are', 'too', 'becomes', 'does', 'a', 'everyone', 'but', 'some', 'regarding', '‘ll', 'against', 'throughout', 'yourselves', 'him', "'d", 'it', 'himself', 'whether', 'move', '’m', 'hereafter', 're', 'while', 'whoever', 'your', 'first', 'amount', 'twelve', 'serious', 'other', 'any', 'off', 'seeming', 'four', 'itself', 'nothing', 'beforehand', 'make', 'out', 'very', 'already', 'various', 'until', 'hers', 'they', 'not', 'them', 'where', 'would', 'since', 'everything', 'at', 'together', 'yet', 'more', 'six', 'back', 'with', 'thereupon', 'becoming', 'around', 'due', 'keep', 'somehow', 'n‘t', 'across', 'all', 'when', 'i', 'empty', 'nine', 'five', 'get', 'see', 'been', 'name', 'between', 'hence', 'ten', 'several', 'from', 'whereupon', 'through', 'hereby', "'ll", 'alone', 'something', 'formerly', 'without', 'above', 'onto', 'except', 'enough', 'become', 'behind', '’d', 'its', 'most', 'n’t', 'might', 'whereas', 'anything', 'if', 'her', 'via', 'fifty', 'is', 'thereby', 'twenty', 'often', 'whereafter', 'their', 'also', 'anyhow', 'cannot', 'our', 'could', 'because', 'who', 'beside', 'by', 'whence', 'being', 'meanwhile', 'this', 'afterwards', 'whenever', 'mostly', 'what', 'one', 'nobody', 'seem', 'less', 'do', '‘d', 'say', 'thus', 'unless', 'along', 'yourself', 'former', 'thru', 'he', 'hundred', 'three', 'sixty', 'me', 'sometime', 'whose', 'you', 'quite', '’ve', 'about', 'even']

In [ ]:
stopwords_fa = ['و', 'در', 'به', 'از', 'كه', 'مي', 'اين', 'است', 'را', 'با', 'هاي', 'براي', 'آن', 'يك', 'شود', 'شده', 'خود', 'ها', 'كرد', 'شد', 'اي', 'تا', 'كند', 'بر', 'بود', 'گفت', 'نيز', 'وي', 'هم', 'كنند', 'دارد', 'ما', 'كرده', 'يا', 'اما', 'بايد', 'دو', 'اند', 'هر', 'خواهد', 'او', 'مورد', 'آنها', 'باشد', 'ديگر', 'مردم', 'نمي', 'بين', 'پيش', 'پس', 'اگر', 'همه', 'صورت', 'يكي', 'هستند', 'بي', 'من', 'دهد', 'هزار', 'نيست', 'استفاده', 'داد', 'داشته', 'راه', 'داشت', 'چه', 'همچنين', 'كردند', 'داده', 'بوده', 'دارند', 'همين', 'ميليون', 'سوي', 'شوند', 'بيشتر', 'بسيار', 'روي', 'گرفته', 'هايي', 'تواند', 'اول', 'نام', 'هيچ', 'چند', 'جديد', 'بيش', 'شدن', 'كردن', 'كنيم', 'نشان', 'حتي', 'اينكه', 'ولی', 'توسط', 'چنين', 'برخي', 'نه', 'ديروز', 'دوم', 'درباره', 'بعد', 'مختلف', 'گيرد', 'شما', 'گفته', 'آنان', 'بار', 'طور', 'گرفت', 'دهند', 'گذاري', 'بسياري', 'طي', 'بودند', 'ميليارد', 'بدون', 'تمام', 'كل', 'تر', 'براساس', 'شدند', 'ترين', 'امروز', 'باشند', 'ندارد', 'چون', 'قابل', 'گويد', 'ديگري', 'همان', 'خواهند', 'قبل', 'آمده', 'اكنون', 'تحت', 'طريق', 'گيري', 'جاي', 'هنوز', 'چرا', 'البته', 'كنيد', 'سازي', 'سوم', 'كنم', 'بلكه', 'زير', 'توانند', 'ضمن', 'فقط', 'بودن', 'حق', 'آيد', 'وقتي', 'اش', 'يابد', 'نخستين', 'مقابل', 'خدمات', 'امسال', 'تاكنون', 'مانند', 'تازه', 'آورد', 'فكر', 'آنچه', 'نخست', 'نشده', 'شايد', 'چهار', 'جريان', 'پنج', 'ساخته', 'زيرا', 'نزديك', 'برداري', 'كسي', 'ريزي', 'رفت', 'گردد', 'مثل', 'آمد', 'ام', 'بهترين', 'دانست', 'كمتر', 'دادن', 'تمامي', 'جلوگيري', 'بيشتري', 'ايم', 'ناشي', 'چيزي', 'آنكه', 'بالا', 'بنابراين', 'ايشان', 'بعضي', 'دادند', 'داشتند', 'برخوردار', 'نخواهد', 'هنگام', 'نبايد', 'غير', 'نبود', 'ديده', 'وگو', 'داريم', 'چگونه', 'بندي', 'خواست', 'فوق', 'ده', 'نوعي', 'هستيم', 'ديگران', 'همچنان', 'سراسر', 'ندارند', 'گروهي', 'سعي', 'روزهاي', 'آنجا', 'يكديگر', 'كردم', 'بيست', 'بروز', 'سپس', 'رفته', 'آورده', 'نمايد', 'باشيم', 'گويند', 'زياد', 'خويش', 'همواره', 'گذاشته', 'شش', 'نداشته', 'شناسي', 'خواهيم', 'آباد', 'داشتن', 'نظير', 'همچون', 'باره', 'نكرده', 'شان', 'سابق', 'هفت', 'دانند', 'جايي', 'بی', 'جز', 'زیرِ', 'رویِ', 'سریِ', 'تویِ', 'جلویِ', 'پیشِ', 'عقبِ', 'بالایِ', 'خارجِ', 'وسطِ', 'بیرونِ', 'سویِ', 'کنارِ', 'پاعینِ', 'نزدِ', 'نزدیکِ', 'دنبالِ', 'حدودِ', 'برابرِ', 'طبقِ', 'مانندِ', 'ضدِّ', 'هنگامِ', 'برایِ', 'مثلِ', 'بارة', 'اثرِ', 'تولِ', 'علّتِ', 'سمتِ', 'عنوانِ', 'قصدِ', 'روب', 'جدا', 'کی', 'که', 'چیست', 'هست', 'کجا', 'کجاست', 'کَی', 'چطور', 'کدام', 'آیا', 'مگر', 'چندین', 'یک', 'چیزی', 'دیگر', 'کسی', 'بعری', 'هیچ', 'چیز', 'جا', 'کس', 'هرگز', 'یا', 'تنها', 'بلکه', 'خیاه', 'بله', 'بلی', 'آره', 'آری', 'مرسی', 'البتّه', 'لطفاً', 'ّه', 'انکه', 'وقتیکه', 'همین', 'پیش', 'مدّتی', 'هنگامی', 'مان', 'تان']

In [ ]:
#df = df[df['text'].str.count(" ") >= 3]

In [ ]:
from random import randint, choices

K_PREDS = 3
K_REPLACE = 3

invalid_word = lambda x: x.isnumeric() or x in string.punctuation or x in stopwords_en or x in stopwords_fa 

def augment_data_batched(examples):
    outputs = []
    masked_sentences = []
    labels = []
    generated_indexs = []
    
    for index, (sentence, label) in enumerate(zip(examples['text'], examples['label'])):
        if not sentence:
            continue
        words = sentence.split(' ')
        
        valids = []
        for i in range(len(words)):
            if not invalid_word(words[i]):
                valids.append(i)

        if len(valids) <= K_REPLACE:
            continue
            
        if len(words) > 200:
            continue
            
        generated_indexs.append(index)

        for K in choices(valids, k=K_REPLACE):
            masked_sentence = " ".join(words[:K]  + ['<mask>'] + words[K+1:])
            masked_sentences.append(masked_sentence)
    
    predictions = pipe(masked_sentences)
    
    for i in range(len(examples['label'])):
        augmented_sequences = []
        
        if i in generated_indexs:
            k = generated_indexs.index(i)
            augmented_sequences = [' '.join(predictions[k][j]["sequence"].split()) for j in range(K_PREDS)]
        
        sequences = [sentence] + augmented_sequences
        
        labels += [examples['label'][i]] * len(sequences)
        outputs += sequences

    return {"text": outputs, "label": labels}

In [ ]:
from tqdm import tqdm

In [ ]:
new_text = []
new_label = []

for i in tqdm(range(0, len(df) - 1, 5)):
    rows = (df.iloc[range(i, i + 5)])
    
    text = tuple(rows.text)
    label = tuple(rows.label)
    
    examples = {'text': text, 'label': label}

    augmented = augment_data_batched(examples)
    new_text += augmented['text']
    new_label += augmented['label']


In [ ]:
augmented = pd.DataFrame({'text': new_text, 'label': new_label}).drop_duplicates(subset=['text'])

In [ ]:
dataset = Dataset.from_pandas(augmented, preserve_index=False).shuffle(seed=42)

In [ ]:
dataset

In [ ]:
dataset.save_to_disk('/kaggle/working/AugmentedFeedbackDataset')

In [ ]:
!zip -1 -r /kaggle/working/AugmentedFeedbackDataset.zip /kaggle/working/AugmentedFeedbackDataset 

In [ ]:
def adjust_labels(batch):
  batch['label_'] = [1 if label == 'spam' else 0 for label in batch['label']]
  return batch

dataset = (
    dataset.map(adjust_labels, batched=True)
    .remove_columns(['label'])
    .rename_column('label_', 'label')
)

In [ ]:
import re

persian_alpha_codepoints = '\u0621-\u0628\u062A-\u063A\u0641-\u0642\u0644-\u0648\u064E-\u0651\u0655\u067E\u0686\u0698\u06A9\u06AF\u06BE\u06CC'

PERSIAN_PATTERN = re.compile('['+persian_alpha_codepoints+']')

def is_persian(example):
    example['is_persian'] = bool(PERSIAN_PATTERN.search(example['text']))
    return example

dataset = dataset.map(is_persian, batched=False)

In [ ]:
plt.figure()
df = dataset.to_pandas()

persian_c, non_persian_c = len(df[df['is_persian'] == 1]), len(df[df['is_persian'] == 0])
plt.bar(0, non_persian_c, label='Non Persian')
plt.bar(1, persian_c, label='Persian')
plt.grid()
plt.xlabel('Class')
plt.ylabel('Count')
plt.legend()
plt.xticks([0, 1], ['Non Persian', 'Persian'], rotation=0)
plt.show()

In [ ]:
plt.figure()

spam_c, ham_c = len(df[df['label'] == 1]), len(df[df['label'] == 0])
plt.bar(0, ham_c, label='Ham')
plt.bar(1, spam_c, label='Spam')
plt.grid()
plt.xlabel('Class')
plt.ylabel('Count')
plt.legend()
plt.xticks([0, 1], ['Ham', 'Spam'], rotation=0)  # The label 0 is for 'Spam' and 1 is for 'Ham'
plt.show()

In [ ]:
ham_to_spam_ratio = ham_c / spam_c
print(f'{ham_to_spam_ratio = }')

In [ ]:
import evaluate

def compute_metrics(eval_pred):
    load_accuracy = evaluate.load("accuracy")
    load_f1 = evaluate.load("f1")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]

    return {"accuracy": accuracy, "f1": f1}

In [ ]:
import torch
with torch.no_grad():
    torch.cuda.empty_cache()

In [ ]:
!pip install gdown

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("/kaggle/working/PersianFinetune")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
model = AutoModelForSequenceClassification.from_pretrained("/kaggle/working/PersianFinetune", num_labels=2)

In [ ]:
def tokenize(batch):
    return tokenizer(batch["text"], truncation=True, max_length=512)

dataset = dataset.map(tokenize, batched=True).train_test_split(0.3)

In [ ]:
train = dataset['train']
test = dataset['test']

In [ ]:
for name, param in model.named_parameters():
    if "class" in name or "layer.5"  in name or "layer.4" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

for name, param in model.named_parameters():
    print(name, param.requires_grad)

In [ ]:
from torch import nn

class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # compute custom loss (suppose one has 2 labels with different weights)
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([1.0, ham_to_spam_ratio], device=model.device))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

In [ ]:
training_args = TrainingArguments(
   output_dir="/kaggle/working/FeedbackFinetune" ,
   learning_rate=1e-5,
   num_train_epochs=8,
   weight_decay=0.001,
   per_device_train_batch_size=64,
   per_device_eval_batch_size=64,
   dataloader_num_workers=2,
   fp16=True,
   warmup_ratio=1/4,
   evaluation_strategy='steps',
   save_total_limit=2,
   save_steps=0.1,
   eval_steps=1/4,
   resume_from_checkpoint=True,
   report_to='none',
   label_smoothing_factor=0.1
)

trainer = WeightedTrainer(
   model=model,
   args=training_args,
   train_dataset=train,
   eval_dataset=test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
trainer.save_model("/kaggle/working/FinalFinetune")

In [ ]:
!zip -1 -r /kaggle/working/FinalFinetune.zip /kaggle/working/FinalFinetune 

In [ ]:
from optimum.intel import OVModelForSequenceClassification
ov_model = OVModelForSequenceClassification.from_pretrained('/kaggle/working/FinalFinetune', export=True)
ov_model.save_pretrained("/kaggle/working/FinalFinetuneOV")

In [ ]:
!zip -r /kaggle/working/FinalFinetuneOV.zip /kaggle/working/FinalFinetuneOV 

In [ ]:
from IPython.display import FileLink, FileLinks
FileLinks('.')